In [161]:
#Using Quantum Computing to solve Linear System of Equations using HLL algorithm

#Loading all required library
import numpy as np
import math
from braket.circuits import Circuit, circuit
from braket.circuits import gates
from braket.devices import LocalSimulator
from braket.circuits import Observable
from datetime import datetime
import pickle


#Library from the amazon braket git
from utils_qft import inverse_qft
from utils_qft import qft

from scipy.linalg import expm

#flag to store the results in pickle file
save_to_pck=False

In [162]:
#Initilize the local simulator
def simulate(circ,shots=1000):
    device = LocalSimulator()
    task = device.run(circ, shots=shots)
    result = task.result()
    
    return result

In [163]:
#Process the result of simulation
def processresult(circ,result):
    # get metadata
    metadata = result.task_metadata
                
    # get measurement results
    measurements = result.measurements
    measured_qubits = result.measured_qubits
    measurement_counts = result.measurement_counts
    measurement_probabilities = result.measurement_probabilities

    # aggregate results
    out = {'circuit': circ,
           'task_metadata': metadata,
           'measurements': measurements,
           'measured_qubits': measured_qubits,
           'measurement_counts': measurement_counts,
           'measurement_probabilities': measurement_probabilities
          }

    if save_to_pck:
        # store results: dump output to pickle with timestamp in filename
        time_now = datetime.strftime(datetime.now(), '%Y%m%d%H%M%S')
        results_file = 'results-'+time_now+'.pck'
        pickle.dump(out, open(results_file, "wb"))
        
    return out

In [164]:
@circuit.subroutine(register=True)
def controlled_unitary(memory,control,lamda,m):
    
    circ=Circuit()
    circ.h(memory)
    circ.rz(memory,2*np.pi*lamda*m)
    circ.cnot(control,memory)
    circ.rz(memory,2*np.pi*lamda*m)
    circ.cnot(control,memory)
    circ.rz(memory,-4*np.pi*lamda*m)
    circ.h(memory)
    return circ



In [165]:
@circuit.subroutine(register=True)
def qft(qubits):
    """
    Construct a circuit object corresponding to the Quantum Fourier Transform (QFT)
    algorithm, applied to the argument qubits.  Does not use recursion to generate the QFT.

    Args:
        qubits (int): The list of qubits on which to apply the QFT
    """
    qftcirc = Circuit()

    # get number of qubits
    num_qubits = len(qubits)

    for k in range(num_qubits):
        # First add a Hadamard gate
        qftcirc.h(qubits[k])

        # Then apply the controlled rotations, with weights (angles) defined by the distance to the control qubit.
        # Start on the qubit after qubit k, and iterate until the end.  When num_qubits==1, this loop does not run.
        for j in range(1,num_qubits - k):
            angle = 2*math.pi/(2**(j+1))
            qftcirc.cphaseshift(qubits[k+j],qubits[k], angle)

    # Then add SWAP gates to reverse the order of the qubits:
    for i in range(math.floor(num_qubits/2)):
        qftcirc.swap(qubits[i], qubits[-i-1])

    return qftcirc


In [166]:
@circuit.subroutine(register=True)
def inverse_qft(qubits):
    """
    Construct a circuit object corresponding to the inverse Quantum Fourier Transform (QFT)
    algorithm, applied to the argument qubits.  Does not use recursion to generate the circuit.

    Args:
        qubits (int): The list of qubits on which to apply the inverse QFT
    """
    # Instantiate circuit object
    qftcirc = Circuit()

    # Fet number of qubits
    num_qubits = len(qubits)

    # First add SWAP gates to reverse the order of the qubits:
    for i in range(math.floor(num_qubits/2)):
        qftcirc.swap(qubits[i], qubits[-i-1])

    # Start on the last qubit and work to the first.
    for k in reversed(range(num_qubits)):

        # Apply the controlled rotations, with weights (angles) defined by the distance to the control qubit.
        # These angles are the negative of the angle used in the QFT.
        # Start on the last qubit and iterate until the qubit after k.
        # When num_qubits==1, this loop does not run.
        for j in reversed(range(1, num_qubits - k)):
            angle = -2*math.pi/(2**(j+1))
            qftcirc.cphaseshift(qubits[k+j],qubits[k], angle)

        # Then add a Hadamard gate
        qftcirc.h(qubits[k])

    return qftcirc

In [167]:
@circuit.subroutine(register=True)
def ancilla_qe(ancilla,qubits, C, t):
    
    circ=Circuit()
    
    N = 2 ** (len(qubits) - 1)
    
    def ancilla_rotation(k):
        if k == 0:
            k = N
        
        theta = 2 * math.asin(C * N * t / (2 * math.pi * k))
        
        return circ.ry(ancilla,theta)
    
    for k in range(1,N+1):
            kGate = ancilla_rotation(k)

            # xor's 1 bits correspond to X gate positions.
            xor = k ^ (k - 1)

            for q in qubits[-2::-1]:
                # Place X gates
                if xor % 2 == 1:
                    circ.x(q)
                xor >>= 1 
            
            return kGate.ccnot(1,2,0)
    
    

In [168]:
'''
Converting a matrix to unitary form using hamiltonian simulation

'''
class HamiltonianSimulation():
    """
    A gate that represents e^iAt.

    This EigenGate + np.linalg.eigh() implementation is used here
    purely for demonstrative purposes. If a large matrix is used,
    the circuit should implement actual Hamiltonian simulation
    """

    def __init__(self, A, t, exponent=1.0):
        self.A = A
        self.t = t
        ws, vs = np.linalg.eigh(A)
        self.eigen_components = []
        for w, v in zip(ws, vs.T):
            theta = w * t / math.pi
            P = np.outer(v, np.conj(v))
            self.eigen_components.append((theta, P))

    def _with_exponent(self, exponent):
        return HamiltonianSimulation(self.A, self.t, exponent)

    def _eigen_components(self):
        return self.eigen_components

In [169]:
#Building HHL Circuit

t = 0.358166 * math.pi    
register_size = 2

# Set C to be the smallest eigenvalue that can be represented by the
# circuit.
C = 2 * math.pi / (2 ** register_size * t)


A = np.array([[1, -1/3], 
              [-1/3, 1]]
            )

w,v=np.linalg.eig(A)

ancilla=0
register=[i for i in range(1,register_size+2)]
hs = HamiltonianSimulation(A, t)

#Quantum Phase Estimation Starts
circ=Circuit()
circ.h(ancilla)
circ.h(register[:-1])

memory=register.pop()
for i, qubit in enumerate(register):
            circ.controlled_unitary(memory,qubit,w[i],i)

circ.inverse_qft(register)                
#Quantum Phase Estimation Ends

#Eigen rotation
circ.ancilla_qe(ancilla,register, C, t)


circ.expectation(observable=Observable.X(), target=1)
circ.expectation(observable=Observable.Y(), target=2)
circ.expectation(observable=Observable.Z(), target=3)

#circ.state_vector()
print(circ)

T  : |0|   1    |2|  3  |4|  5   |6|7|   8    |9|   10   |11|   12    |13|     14     |15|16|17| Result Types |
                                                                                                               
q0 : -H-Ry(1.05)-----------------------------------------------------------------------------X-----------------
                                                                                             |                 
q1 : -H----------C-------C-----------------------------------SWAP---------PHASE(-1.57)-H--X--C--Expectation(X)-
                 |       |                                   |            |                  |                 
q2 : -H----------|-------|---------------------C----------C--SWAP------H--C------------------C--Expectation(Y)-
                 |       |                     |          |                                                    
q3 : -H-Rz(0)----X-Rz(0)-X-Rz(-0)-H-H-Rz(4.19)-X-Rz(4.19)-X--Rz(-8.38)-H------------------------Expectat

In [170]:
#Running HHL circuit
result=simulate(circ)
out=processresult(circ,result)

#print(out)
print(result.values)
print(result.measurements)

'''

print(list(result))
for label, result in zip(('X', 'Y', 'Z'), list(result)):
    expectation = 1 - 2 * np.mean(result.measurements[:,0])
    print('{} = {}'.format(label, expectation))
        
'''

[0.0, 0.026, 0.24]
[[1 1 1 1]
 [1 1 1 1]
 [1 0 1 0]
 ...
 [1 1 0 0]
 [0 0 1 0]
 [1 1 0 1]]


"\n\nprint(list(result))\nfor label, result in zip(('X', 'Y', 'Z'), list(result)):\n    expectation = 1 - 2 * np.mean(result.measurements[:,0])\n    print('{} = {}'.format(label, expectation))\n        \n"